In [1]:
import numpy as np
import pandas as pd

# melt --> wide to long format
### wide --> horizontal view with more columns
### long --> dbr view with more rows

In [3]:
# Following data is in wide format
df1 = pd.DataFrame({'cse':[100,150],
                  'ece':[150,170],
                  'mech': [200,220]},
                  index = [2019,2020])
print(df1)

      cse  ece  mech
2019  100  150   200
2020  150  170   220


In [5]:
df1.melt()

,variable,value
0,cse,100
1,cse,150
2,ece,150
3,ece,170
4,mech,200
5,mech,220


In [9]:
df2 = df1.reset_index(names = 'year')

In [10]:
df2

,year,cse,ece,mech
0,2019,100,150,200
1,2020,150,170,220


In [11]:
df2.melt()

,variable,value
0,year,2019
1,year,2020
2,cse,100
3,cse,150
4,ece,150
5,ece,170
6,mech,200
7,mech,220


### We don't want year to be converted to rows

In [12]:
df2.melt(id_vars='year')

,year,variable,value
0,2019,cse,100
1,2020,cse,150
2,2019,ece,150
3,2020,ece,170
4,2019,mech,200
5,2020,mech,220


In [13]:
df2.melt(id_vars='year',var_name='branch',value_name='num_students')

,year,branch,num_students
0,2019,cse,100
1,2020,cse,150
2,2019,ece,150
3,2020,ece,170
4,2019,mech,200
5,2020,mech,220


## Loading Covid Dataset

In [29]:
confirmed = pd.read_csv('data/time_series_covid19_confirmed_global.csv')
death = pd.read_csv('data/time_series_covid19_deaths_global.csv')

In [30]:
confirmed.head(2)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,12/24/22,12/25/22,12/26/22,12/27/22,12/28/22,12/29/22,12/30/22,12/31/22,1/1/23,1/2/23
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,207310,207399,207438,207460,207493,207511,207550,207559,207616,207627
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,333749,333749,333751,333751,333776,333776,333806,333806,333811,333812


In [31]:
death.head(2)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,12/24/22,12/25/22,12/26/22,12/27/22,12/28/22,12/29/22,12/30/22,12/31/22,1/1/23,1/2/23
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,7845,7846,7846,7846,7846,7847,7847,7849,7849,7849
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,3595,3595,3595,3595,3595,3595,3595,3595,3595,3595


### Both are organised in wide formats
### We want to have columns country, date,confirm,death

In [32]:
confirmed_dbr = confirmed.melt(id_vars= ['Province/State', 'Country/Region', 'Lat', 'Long'],
                              var_name='Date', value_name= 'nos_confirmed')

In [33]:
confirmed_dbr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 311253 entries, 0 to 311252
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   Province/State  98007 non-null   object 
 1   Country/Region  311253 non-null  object 
 2   Lat             309099 non-null  float64
 3   Long            309099 non-null  float64
 4   Date            311253 non-null  object 
 5   nos_confirmed   311253 non-null  int64  
dtypes: float64(2), int64(1), object(3)
memory usage: 14.2+ MB


In [34]:
death_dbr = death.melt(id_vars= ['Province/State', 'Country/Region', 'Lat', 'Long'],
                              var_name='Date', value_name= 'nos_death')

In [35]:
death_dbr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 311253 entries, 0 to 311252
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   Province/State  98007 non-null   object 
 1   Country/Region  311253 non-null  object 
 2   Lat             309099 non-null  float64
 3   Long            309099 non-null  float64
 4   Date            311253 non-null  object 
 5   nos_death       311253 non-null  int64  
dtypes: float64(2), int64(1), object(3)
memory usage: 14.2+ MB


In [36]:
# cols_to_drop = ['Province/State','Lat','Long']
# death_dbr.drop(columns=cols_to_drop,inplace=True)
# confirmed_dbr.drop(columns=cols_to_drop,inplace=True)

In [40]:
# Merge both tables
merge_df = confirmed_dbr.merge(death_dbr, on = ['Province/State','Lat','Long','Country/Region', 'Date'],how = 'inner')[['Country/Region','Date','nos_confirmed','nos_death']]

In [41]:
merge_df

,Country/Region,Date,nos_confirmed,nos_death
0,Afghanistan,1/22/20,0,0
1,Albania,1/22/20,0,0
2,Algeria,1/22/20,0,0
3,Andorra,1/22/20,0,0
4,Angola,1/22/20,0,0
...,...,...,...,...
311248,West Bank and Gaza,1/2/23,703228,5708
311249,Winter Olympics 2022,1/2/23,535,0
311250,Yemen,1/2/23,11945,2159
311251,Zambia,1/2/23,334661,4024


In [42]:
merge_df[merge_df['Country/Region']=='Afghanistan']

,Country/Region,Date,nos_confirmed,nos_death
0,Afghanistan,1/22/20,0,0
289,Afghanistan,1/23/20,0,0
578,Afghanistan,1/24/20,0,0
867,Afghanistan,1/25/20,0,0
1156,Afghanistan,1/26/20,0,0
...,...,...,...,...
309808,Afghanistan,12/29/22,207511,7847
310097,Afghanistan,12/30/22,207550,7847
310386,Afghanistan,12/31/22,207559,7849
310675,Afghanistan,1/1/23,207616,7849


# Pivot Table

In [43]:
import seaborn as sns

In [44]:
tips = sns.load_dataset('tips')
tips.head(3)

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3


In [45]:
tips.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   total_bill  244 non-null    float64 
 1   tip         244 non-null    float64 
 2   sex         244 non-null    category
 3   smoker      244 non-null    category
 4   day         244 non-null    category
 5   time        244 non-null    category
 6   size        244 non-null    int64   
dtypes: category(4), float64(2), int64(1)
memory usage: 7.4 KB


In [46]:
tips.describe()

,total_bill,tip,size
count,244.000000,244.000000,244.000000
mean,19.785943,2.998279,2.569672
std,8.902412,1.383638,0.951100
min,3.070000,1.000000,1.000000
25%,13.347500,2.000000,2.000000
50%,17.795000,2.900000,2.000000
75%,24.127500,3.562500,3.000000
max,50.810000,10.000000,6.000000


## We want to check how the average tip vary with sex and smoking

In [48]:
tips.pivot_table(index = 'sex',columns=['smoker'],values = ['total_bill','tip'],aggfunc=['mean'])

mean                                
             tip           total_bill           
smoker       Yes        No        Yes         No
sex                                             
Male    3.051167  3.113402  22.284500  19.791237
Female  2.931515  2.773519  17.977879  18.105185

In [50]:
tips.pivot_table(index = 'sex',columns=['smoker'],values = ['total_bill','tip'],aggfunc={'total_bill':'median',
                                                                                               'tip':'mean'
                                                                                               })

tip           total_bill       
smoker       Yes        No        Yes     No
sex                                         
Male    3.051167  3.113402      20.39  18.24
Female  2.931515  2.773519      16.27  16.69

In [51]:
tips.pivot_table(index = 'sex',columns=['smoker'],values = ['total_bill','tip'],aggfunc={'total_bill':'median',
                                                                                               'tip':'mean'
                                                                                               },margins = True)

tip                     total_bill               
smoker       Yes        No       All        Yes     No     All
sex                                                           
Male    3.051167  3.113402  3.089618      20.39  18.24  18.350
Female  2.931515  2.773519  2.833448      16.27  16.69  16.400
All     3.008710  2.991854  2.998279      17.92  17.59  17.795